In [26]:
# Notebook 05: The Backtester (Thesis Validation)
#
# **Purpose:** To validate the "Intelligence Desk" thesis by backtesting
# the signals generated and logged in `trade_journal.csv`.
#
# **Methodology:** "Model C: Relative-Value (Pairs) Proxy"
#
# This backtest does *not* simulate complex options Greeks. Instead, it
# tests the core *directional hypothesis* of your trades.
#
# 1.  **Load `trade_journal.csv`:** This is our list of trade signals.
# 2.  **Load 5-Years of Historical Data:** Re-create the master dataset
#     of prices, drivers, and (most importantly) the *historical Z-Score
#     spreads* (`GLD_TLT_Spread_Norm`, etc.).
# 3.  **Simulate P&L:** For each trade in the journal:
#     * We find the `Entry_ZScore` of the relevant spread on the trade date.
#     * We determine the trade direction (e.g., "Long GLD / Short BNO" =
#         LONG the `GLD_BNO_Spread_Norm`).
#     * We hold for a fixed `HOLDING_PERIOD` (e.g., 30 days).
#     * We find the `Exit_ZScore` at the end of the period.
#     * The P&L is calculated from the *change in the Z-Score*.
#
# **Result:** This directly answers: "Is my analysis of these relationships
# (Sovereign Risk, Recession, etc.) profitable over time?"

In [27]:
# Cell 1: Imports
#
# Import all libraries needed for data, analysis, and visualization.
import yfinance as yf
from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import zscore
import requests
import os
import json
from datetime import datetime, timedelta
import csv
import math

# Configure Pandas for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Cell 1: Libraries imported successfully.")

✅ Cell 1: Libraries imported successfully.


In [28]:
# Cell 2: API Configuration & Backtest Parameters
#
# Set up API keys and the core parameters for the backtest.

# --- FRED API Configuration ---
os.environ.setdefault("FRED_API_KEY", "24a7e70f4690fbab7d2571d335107c68") # ❗️ Set your key

# --- LLM Configuration (for final analysis) ---
os.environ.setdefault("LLM_BASE_URL", "http://127.0.0.1:11434")
os.environ.setdefault("LLM_MODEL", "qwen2.5:7b")

# --- Backtest Parameters ---
# This is our assumed holding period for a "swing trade"
HOLDING_PERIOD_DAYS = 30 
# This is a multiplier to turn Z-Score "points" into a dollar value
# (e.g., a 0.5 Z-Score move = $50 P&L per contract)
Z_SCORE_POINT_VALUE = 100 

# --- Trade Journal File ---
JOURNAL_FILE = 'quant/trade_journal.csv'

# --- Tickers for full historical data ---
ALL_YFINANCE_TICKERS = ["GLD", "TLT", "BNO", "FXF", "FXE"]
ALL_FRED_SERIES_IDS = [
    "DFII10", "T10YIE", "DTWEXBGS", "GVZCLS", "VIXCLS", 
    "OVXCLS", "IPMAN", "CPILFESL", "IRLTLT01ITM156N", 
    "IRLTLT01DEM156N", "CPALTT01CHM657N", "CP0000EZ19M086NEST", "VIXCLS"
]

# --- Date Range for Historical Data ---
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=5 * 365) # 5 years of data

print("✅ Cell 2: Configuration loaded.")

✅ Cell 2: Configuration loaded.


In [29]:
# Cell 3: Data Collection - yfinance, FRED, and SNB
#
# This cell creates our "Oracle" DataFrame: a complete historical
# dataset of all assets and drivers for the 5-year period.

try:
    print("Fetching all historical data...")
    # 1. Fetch yfinance
    yf_data = yf.download(ALL_YFINANCE_TICKERS, start=START_DATE, end=END_DATE)
    df_yf = yf_data.xs('Close', level=0, axis=1).copy()
    df_yf.index = pd.to_datetime(df_yf.index)

    # 2. Fetch FRED
    fred = Fred(api_key=os.getenv("FRED_API_KEY"))
    df_fred_list = []
    for series_id in ALL_FRED_SERIES_IDS:
        series = fred.get_series(series_id, START_DATE, END_DATE)
        df_fred_list.append(series.rename(series_id))
    df_fred = pd.concat(df_fred_list, axis=1)
    df_fred.index = pd.to_datetime(df_fred.index)

    # 3. Fetch SNB (Copy-pasted from Notebook 04)
    snb_url = "https://data.snb.ch/api/cube/snbgwdchfsgw/data/json/en"
    params = {"dimSel": "D0(GI,UEB,TG)", "fromDate": START_DATE.strftime('%Y-%m-%d'), "toDate": END_DATE.strftime('%Y-%m-%d')}
    response = requests.get(snb_url, params=params)
    snb_data = response.json()
    total_sight_deposits = None
    for ts in snb_data["timeseries"]:
        if ts["header"][0]["dimItem"] == "Total sight deposits in Swiss francs at the SNB":
            total_sight_deposits = ts
            break
    df_snb = pd.DataFrame(total_sight_deposits["values"])
    df_snb["date"] = pd.to_datetime(df_snb["date"])
    df_snb["value"] = pd.to_numeric(df_snb["value"], errors="coerce")
    df_snb = df_snb.set_index("date").rename(columns={"value": "SNB_Sight_Deposits"})

    print("✅ All data fetched.")

except Exception as e:
    print(f"❌ Cell 3: Data fetch failed: {e}")

Fetching all historical data...


/tmp/ipykernel_1127242/3497980138.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf_data = yf.download(ALL_YFINANCE_TICKERS, start=START_DATE, end=END_DATE)
[                       0%                       ]

[*********************100%***********************]  5 of 5 completed


✅ All data fetched.


In [30]:
# Cell 4: Data Consolidation & Spread Calculation
#
# Merge all data into a single DataFrame, clean it, and
# re-calculate the *historical time series* for our four Z-Score spreads.

try:
    print("Consolidating and cleaning data...")
    # Merge all three data sources
    df_hist = pd.merge(df_yf, df_fred, left_index=True, right_index=True, how='outer')
    df_hist = pd.merge(df_hist, df_snb, left_index=True, right_index=True, how='outer')
    
    # Fill all missing data (weekends, holidays)
    df_hist = df_hist.ffill()
    df_hist = df_hist.dropna(subset=ALL_YFINANCE_TICKERS) # Ensure we have price data

    print("Calculating historical Z-Scores...")
    # Normalize all columns
    # We must drop non-numeric columns if any exist (e.g., from bad API pulls)
    df_numeric = df_hist.select_dtypes(include=[np.number])
    df_normalized = df_numeric.apply(zscore)

    # Calculate the 4 historical Z-Score Spreads
    df_hist['GLD_TLT_Spread_Norm'] = df_normalized['GLD'] - df_normalized['TLT']
    df_hist['GLD_BNO_Spread_Norm'] = df_normalized['GLD'] - df_normalized['BNO']
    df_hist['CHF_EUR_Spread_Norm'] = df_normalized['FXF'] - df_normalized['FXE']
    df_hist['CHF_GLD_Spread_Norm'] = df_normalized['FXF'] - df_normalized['GLD']
    
    # Final cleanup of any NaNs created by Z-Score
    df_hist = df_hist.dropna()

    print("✅ Cell 4: Historical Data Oracle created successfully.")
    print(df_hist[['GLD_TLT_Spread_Norm', 'GLD_BNO_Spread_Norm', 'CHF_EUR_Spread_Norm', 'CHF_GLD_Spread_Norm']].tail())

except Exception as e:
    print(f"❌ Cell 4: Data processing failed: {e}")

Consolidating and cleaning data...
Calculating historical Z-Scores...
✅ Cell 4: Historical Data Oracle created successfully.
            GLD_TLT_Spread_Norm  GLD_BNO_Spread_Norm  CHF_EUR_Spread_Norm  CHF_GLD_Spread_Norm
2025-10-28             3.588412             2.636908             1.042067            -0.646282
2025-10-29             3.615298             2.564096             0.918247            -0.827459
2025-10-30             3.783354             2.717645             0.921313            -1.018513
2025-10-31             3.760110             2.622019             0.909290            -1.051744
2025-11-03             3.804490             2.616989             0.843446            -1.150889


In [31]:
# Cell 5: Load Trade Journals (Corrected & Robust)
#
# Load all 4 individual trade journals, standardize their columns,
# and concatenate them into one master DataFrame for backtesting.

# --- List of all our journal files ---
JOURNAL_FILES = [
    'quant/journal_zb_gc.csv',
    'quant/journal_bz_gc.csv',
    'quant/journal_chf_eur.csv',
    'quant/journal_chf_gc.csv'
]

all_journals = []

try:
    print("Loading and standardizing all trade journals...")
    
    for file in JOURNAL_FILES:
        # Check if file exists and is not empty
        if not os.path.isfile(file) or os.path.getsize(file) == 0:
            print(f"Info: '{file}' not found or is empty. Skipping.")
            continue
            
        df_log = pd.read_csv(file, parse_dates=['Date'])
        
        # --- Standardize Columns ---
        # This is the key: we create a standard format.
        df_std = pd.DataFrame()
        df_std['Date'] = df_log['Date']
        df_std['Pair'] = df_log['Pair']
        
        # Check for single-leg vs. pair-trade notebooks
        if 'Strategy' in df_log.columns:
            # This is a single-leg trade (ZB/GC or CHF/EUR)
            df_std['Strategy'] = df_log['Strategy']
            df_std['Contracts'] = df_log['Contracts_Sized']
            df_std['Directional_Bias'] = df_log.get('Target_Asset') # Use Target_Asset as bias
        else:
            # This is a two-leg trade (BZ/GC or CHF/GC)
            # We will test the "first leg" as the primary signal
            df_std['Strategy'] = df_log['Strategy_GLD'] # (e.g., test the GLD leg)
            df_std['Contracts'] = df_log['Contracts_GLD']
            df_std['Directional_Bias'] = df_log.get('Directional_Bias')

        # Add the all-important Z-Score signal
        df_std['Signal_ZScore'] = df_log['Signal_ZScore']

        all_journals.append(df_std)

    if not all_journals:
        raise FileNotFoundError("No journal files found or all journals are empty. Please run Notebooks 1-4.")
        
    # Concatenate all standardized DataFrames
    df_journal = pd.concat(all_journals).sort_values('Date').reset_index(drop=True)

    # Filter out any 'No_Trade' signals
    df_journal = df_journal[
        (df_journal['Strategy'].fillna('No_Trade') != 'No_Trade')
    ].dropna(subset=['Date']).reset_index(drop=True)

    print(f"✅ Cell 5: Loaded and combined {len(df_journal)} total trade signals.")
    print(df_journal.tail())

except FileNotFoundError as e:
    print(f"❌ Cell 5: {e}")
    df_journal = pd.DataFrame() # Create empty df
except Exception as e:
    print(f"❌ Cell 5: An unexpected error occurred: {e}")
    df_journal = pd.DataFrame() # Create empty df

Loading and standardizing all trade journals...
✅ Cell 5: Loaded and combined 4 total trade signals.
        Date     Pair          Strategy  Contracts      Directional_Bias  Signal_ZScore
0 2025-11-03    ZB_GC   Bull_Put_Spread          1                   GLD       3.800889
1 2025-11-03    BZ_GC  Bear_Call_Spread          1  Long GLD / Short BNO       2.616989
2 2025-11-03  CHF_EUR       Iron_Condor          1   FXE (via Short EUR)       0.843446
3 2025-11-03   CHF_GC   Calendar_Spread          1  Long GLD / Short FXF      -1.150016


In [32]:
# Cell 6: Helper Function - Get Spread Details (Corrected)
#
# This function reads a row from the *standardized* trade journal and
# determines which Z-Score spread we are trading and in which direction.
#
# FIX: This function now reads from the 'Contracts' column, which was
# standardized in Cell 5.

def get_spread_details(trade_row):
    """
    Parses a trade journal row to find the spread name and trade direction.
    Returns: (spread_name, direction, contracts)
    """
    pair = trade_row['Pair']
    
    # --- THIS IS THE FIX ---
    # The standardized column is 'Contracts' for ALL rows.
    contracts = trade_row['Contracts'] 
    
    if pair == 'ZB_GC':
        # ZB/GC: Bias is from the strategy name (e.g., Bull_Put_Spread)
        direction = 1 if 'Bull' in trade_row['Strategy'] else -1
        return 'GLD_TLT_Spread_Norm', direction, contracts
    
    elif pair == 'BZ_GC':
        # BZ/GC: Bias is from the 'Directional_Bias' column
        direction = 1 if 'Long GLD' in trade_row['Directional_Bias'] else -1
        return 'GLD_BNO_Spread_Norm', direction, contracts
    
    elif pair == 'CHF_EUR':
        # CHF/EUR: Bias is from the 'Directional_Bias' column
        direction = 1 if 'Bearish' in trade_row['Directional_Bias'] else -1
        return 'CHF_EUR_Spread_Norm', direction, contracts
    
    elif pair == 'CHF_GC':
        # CHF/GC: Bias is from the 'Directional_Bias' column
        direction = -1 if 'Long GLD' in trade_row['Directional_Bias'] else 1
        return 'CHF_GLD_Spread_Norm', direction, contracts
    
    else:
        print(f"Warning: Unknown pair '{pair}' in journal.")
        return None, 0, 0

print("✅ Cell 6: Helper function defined.")

✅ Cell 6: Helper function defined.


In [33]:
# Cell 7: The Backtest Loop (Corrected)
#
# This is the core of the backtest. We iterate through every trade
# in our journal and simulate the P&L.
#
# --- THIS IS THE FIX ---
# We add a "guard clause" to check if the trade's Exit_Date
# is *after* our last available data point. If it is, we skip it
# because the trade is still open (or in the future).

try:
    print("Running backtest loop...")
    results = []
    
    # Get the last date of *real* data we have
    last_data_date = df_hist.index.max()
    print(f"Historical data loaded. Last available date is: {last_data_date.strftime('%Y-%m-%d')}")

    for _, row in df_journal.iterrows():
        entry_date = row['Date']
        exit_date = entry_date + pd.Timedelta(days=HOLDING_PERIOD_DAYS)
        
        # --- THE FIX IS HERE ---
        # If the calculated exit date is after our last data point,
        # we cannot backtest it. Skip this trade.
        if exit_date > last_data_date:
            print(f"  -> Skipping trade from {entry_date.strftime('%Y-%m-%d')}: Exit date {exit_date.strftime('%Y-%m-%d')} is in the future.")
            continue
            
        # Get spread details from our helper
        spread_name, direction, contracts = get_spread_details(row)
        
        if spread_name is None or spread_name not in df_hist.columns:
            print(f"  -> Warning: Skipping trade on {entry_date.date()} for pair {row['Pair']}. Invalid spread name.")
            continue
            
        # Find entry Z-Score
        # Use .loc, but get the value as of entry_date or the *next* available day
        entry_z = df_hist.loc[df_hist.index.asof(entry_date), spread_name]
        
        # Find exit Z-Score
        # Use .loc, but get the value as of exit_date or the *next* available day
        exit_z = df_hist.loc[df_hist.index.asof(exit_date), spread_name]
            
        # Calculate P&L
        # P&L = (Change in Z-Score) * Direction * Contracts * Value per Point
        pnl = (exit_z - entry_z) * direction * contracts * Z_SCORE_POINT_VALUE
        
        results.append({
            'Entry_Date': entry_date,
            'Exit_Date': exit_date,
            'Pair': row['Pair'],
            'Directional_Bias': row.get('Directional_Bias', row.get('Strategy')),
            'Entry_ZScore': entry_z,
            'Exit_ZScore': exit_z,
            'PnL': pnl,
            'Contracts': contracts
        })

    if not results:
        print("\nNo completed trades found to backtest. All signals may be too recent.")
    
    df_results = pd.DataFrame(results)
    print(f"\n✅ Cell 7: Backtest complete. {len(df_results)} completed trades were simulated.")
    
    if not df_results.empty:
        print(df_results.tail())

except Exception as e:
    print(f"❌ Cell 7: Backtest loop failed: {e}")

Running backtest loop...
Historical data loaded. Last available date is: 2025-11-03
  -> Skipping trade from 2025-11-03: Exit date 2025-12-03 is in the future.
  -> Skipping trade from 2025-11-03: Exit date 2025-12-03 is in the future.
  -> Skipping trade from 2025-11-03: Exit date 2025-12-03 is in the future.
  -> Skipping trade from 2025-11-03: Exit date 2025-12-03 is in the future.

No completed trades found to backtest. All signals may be too recent.

✅ Cell 7: Backtest complete. 0 completed trades were simulated.


In [34]:
# Cell 8: Performance Analysis & Visualization
#
# Calculate and display the final performance metrics and equity curve.

try:
    print("Calculating performance metrics...")
    
    total_pnl = df_results['PnL'].sum()
    num_trades = len(df_results)
    win_rate = (df_results['PnL'] > 0).mean() * 100
    avg_pnl = df_results['PnL'].mean()
    avg_win = df_results[df_results['PnL'] > 0]['PnL'].mean()
    avg_loss = df_results[df_results['PnL'] < 0]['PnL'].mean()
    
    # Calculate Sharpe Ratio
    # We are simulating ~12 trades per year (30-day holds)
    ann_factor = np.sqrt(252 / HOLDING_PERIOD_DAYS)
    sharpe_ratio = (df_results['PnL'].mean() / df_results['PnL'].std()) * ann_factor
    
    # Calculate Drawdown
    df_results['Equity_Curve'] = df_results['PnL'].cumsum()
    df_results['Running_Max'] = df_results['Equity_Curve'].cummax()
    df_results['Drawdown'] = df_results['Equity_Curve'] - df_results['Running_Max']
    max_drawdown = df_results['Drawdown'].min()
    
    # --- Print Performance Report ---
    print("\n--- 📈 BACKTEST PERFORMANCE REPORT ---")
    print(f"Total P&L:         ${total_pnl:,.2f}")
    print(f"Total Trades:      {num_trades}")
    print(f"Win Rate:          {win_rate:.2f}%")
    print(f"Avg. P&L / Trade:  ${avg_pnl:,.2f}")
    print(f"Avg. Win / Trade:  ${avg_win:,.2f}")
    print(f"Avg. Loss / Trade: ${avg_loss:,.2f}")
    print(f"Sharpe Ratio:      {sharpe_ratio:.2f}")
    print(f"Max Drawdown:      ${max_drawdown:,.2f}")
    print("--------------------------------------")
    
    # --- Plot Equity Curve ---
    plt.figure(figsize=(15, 7))
    df_results.set_index('Entry_Date')['Equity_Curve'].plot(
        title=f"Strategy Equity Curve (Z-Score Proxy Backtest)\nSharpe: {sharpe_ratio:.2f} | Max DD: ${max_drawdown:,.2f}",
        color='blue',
        linewidth=2
    )
    plt.ylabel("Cumulative P&L ($)")
    plt.xlabel("Date")
    plt.grid(True, linestyle=':')
    plt.legend(["Cumulative P&L"])
    plt.tight_layout()
    plt.show()

    # Save metrics for final cell
    backtest_summary = f"""
- Total P&L: ${total_pnl:,.2f}
- Total Trades: {num_trades}
- Win Rate: {win_rate:.2f}%
- Sharpe Ratio: {sharpe_ratio:.2f}
- Max Drawdown: ${max_drawdown:,.2f}
    """

except Exception as e:
    print(f"❌ Cell 8: Performance analysis failed: {e}")
    backtest_summary = "Backtest failed to produce metrics."

Calculating performance metrics...
❌ Cell 8: Performance analysis failed: 'PnL'


In [35]:
# Cell 9: LLM Backtest Critique
#
# The final step: Feed the performance report to the LLM and ask it
# to critique the strategy, per your thesis.

# --- Helper function (copy from other notebooks) ---
def ask_llm(prompt: str, model=None, temperature=0.1):
    model_to_use = model or os.getenv("LLM_MODEL")
    url_to_use = f"{os.getenv('LLM_BASE_URL')}/api/chat"
    chat_payload = {"model": model_to_use, "messages": [{"role": "user", "content": prompt}], "options": {"temperature": temperature}, "stream": False}
    try:
        response = requests.post(url_to_use, json=chat_payload)
        response.raise_for_status()
        json_response = response.json()
        return json_response["message"]["content"]
    except Exception as e:
        print(f"❌ LLM Error: {e}")
        return "Error connecting to LLM."

try:
    print("Sending backtest results to LLM for critique...")
    
    # --- Create the LLM Prompt ---
    prompt = f"""
**Role:** You are an Independent Intelligence Desk Analyst, reviewing the
performance of your new "Z-Score RV" strategy.

**Thesis:** My strategy was to identify RV opportunities in 4 pairs
(ZB/GC, BZ/GC, CHF/EUR, CHF/GC), find a thematic/volatility setup using
an LLM, and then trade the Z-Score of the spread with a 30-day hold.

**Backtest Results (Z-Score P&L Proxy):**
{backtest_summary}

**Your Task (Provide a 3-bullet synthesis):**
1.  **Performance Critique:** Based on the metrics (especially the Sharpe
    Ratio and Max Drawdown), is this a viable, profitable edge, or is
    it just noise?
2.  **Biggest Flaw:** What is the most likely flaw in this strategy or
    backtest? (e.g., "The model is only 30-day holds and may be
    missing mean-reversion," "The Z-Score P&L is not realistic,"
    "The win rate is too low.")
3.  **Next Step:** What is the #1 thing I should do to improve this?
    (e.g., "Test a different holding period," "Move from Z-Score
    backtest to a real options backtest (Model A/B)," "Filter for
    higher Z-Score signals > 2.0.")
"""
    
    # Get LLM response
    llm_critique = ask_llm(prompt)
    
    print("\n--- 🤖 LLM STRATEGY CRITIQUE ---")
    print(llm_critique)
    print("---------------------------------")

except Exception as e:
    print(f"❌ Cell 9: LLM Critique failed: {e}")

Sending backtest results to LLM for critique...



--- 🤖 LLM STRATEGY CRITIQUE ---
### Synthesis

1. **Performance Critique:**
   - Based on the metrics, particularly the absence of Sharpe Ratio and Max Drawdown values in your backtest results, it appears that this strategy may not have produced a viable or profitable edge. The lack of performance metrics suggests that either the strategy did not generate consistent returns over time or there might be significant issues with the data or model implementation.

2. **Biggest Flaw:**
   - The most likely flaw in this strategy or backtest is that the Z-Score P&L proxy may not accurately reflect real-world trading performance. Z-Score calculations are based on historical data and assume normal distributions, which might not hold true for financial markets characterized by non-normal distributions and significant volatility.

3. **Next Step:**
   - The #1 thing you should do to improve this strategy is to validate the Z-Score calculation method against real trading outcomes. Start by impleme